# Explore here

It's recommended to use this notebook for exploration purposes.

For example: 

1. You could import the CSV generated by python into your notebook and explore it.
2. You could connect to your database using `pandas.read_sql` from this notebook and explore it.

In [ ]:
# Example reading the SQL database from here

# from dotenv import load_dotenv
# from sqlalchemy import create_engine
# import pandas as pd

# # load the .env file variables
# load_dotenv()

# connection_string = f"mysql+pymysql://{os.getenv('DB_USER')}:{os.getenv('DB_PASSWORD')}@{os.getenv('DB_HOST')}/{os.getenv('DB_NAME')}?autocommit=true"
# engine = create_engine(connection_string)
# engine.connect()

# dataframe = pd.read_sql("Select * from books;", engine)
# print(dataframe.describe())

In [ ]:
# Example importing the CSV here

# dataframe = pd.read_csv('../path/to/file.csv')
# dataframe.describe()

In [ ]:
url = "https://www.macrotrends.net/stocks/charts/TSLA/tesla/revenue"

import requests
pag = requests.get(url)

In [ ]:
print(pag)
# si es 200 es OK

In [ ]:
pag_texto = pag.text
pag_texto

In [ ]:
from bs4 import BeautifulSoup

In [ ]:
text_beaut = BeautifulSoup(pag_texto, "html.parser")
text_beaut

In [ ]:
tablas = text_beaut.findAll("table")
print(len(tablas))

In [ ]:
id_tabla_quarter = None

for i in range(len(tablas)):
    if "Tesla Quarterly Revenue" in str(tablas[i]):
        id_tabla_quarter = i
        print("La tabla es la número", id_tabla_quarter)
        break

In [ ]:
tabla_quarter_body = tabla_quarter.tbody
tabla_quarter_body

In [ ]:
lista_tr = tabla_quarter_body.find_all("tr") # el objeto fin_all devuelve una lista
lista_tr

In [ ]:
for tr in lista_tr:
    date= tr.find_all("td")
    print("*"*10)
    print(len(date), date)
    print(date[0].text, date[1].text)

In [ ]:
revenue_ls = []

from tqdm.auto import tqdm # libreria que me permite saber en que elemento está dentro de todo el ciclo for y cuál es su tiempo. Además carga la barra de forma iterativa

for tr in tqdm(lista_tr):
    all_tr = tr.find_all("td")
    date = all_tr[0].text
    revenue = all_tr[1].text 
    revenue_ls.append([date,revenue])
print(revenue_ls)

In [ ]:
import pandas as pd

revenue_df = pd.DataFrame(revenue_ls, columns=["Date", "Revenue"])
revenue_df.head()

In [ ]:
elemento = revenue_df["Revenue"][0]
elemento = elemento.replace("$","")
elemento = elemento.replace(",","")
elemento

In [ ]:
import numpy as np

def preproc_revenue(texto):
    texto = texto.replace("$","")
    texto = texto.replace(",","")
    if texto == "":
        return np.nan
    return float(texto)

In [ ]:
revenue_df["Revenue"]=revenue_df["Revenue"].apply(preproc_revenue) # aplico la función a mi columna que le hago el cambio
revenue_df

In [ ]:
revenue_df = revenue_df.dropna(subset="Revenue")

In [ ]:
revenue_df.to_csv("revenue_df.csv", index=None)

## Generar tabla SQL

In [ ]:
import sqlite3

In [ ]:
connection = sqlite3.Connection("Tesla.db")

c = connection.cursor()

# Create table
c.execute('''CREATE TABLE revenue
         (Date, Revenue)''')

records = revenue_df.to_records(index=False)
list_of_tuples = list(records)


In [ ]:
c.executemany('INSERT INTO revenue (Date, Revenue) values (2022-06-16,5000)', list_of_tuples) # Inserto valores

connection.commit()